In [ ]:
# Import required libraries

import mdtraj as md
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sb

In [2]:
## Define distance calc function

def calc_com_distance(traj):

    # Compute center of mass for the first group (chain A)
    chaina_com = md.compute_center_of_mass(traj, select='(residue 2174 or residue 2176 or residue 2177 or residue 2178 or residue 2179 or residue 2180 or residue 2181 or residue 2182 or residue 2183) and chainid 0 and backbone')

    # Compute center of mass for the second group (chain B)
    chainb_com = md.compute_center_of_mass(traj, select='(residue 2579 or residue 2580 or residue 2581 or residue 2582 or residue 2583 or residue 2584 or residue 2585) and chainid 1 and backbone')

    # Compute Euclidean distance between centers of mass at each frame
    distances = np.linalg.norm(chainb_com - chaina_com, axis=1)
    
    return distances


In [3]:

def trim_replicate_timeseries(data, total_ns, n_replicates, trim_ns):

    data = np.asarray(data)
    n_frames = data.shape[0]
    frames_per_rep = n_frames // n_replicates

    # assume each replicate represents 500 ns (as in your system)
    frames_per_ns = frames_per_rep / 500.0
    trim_frames = int(trim_ns * frames_per_ns)

    trimmed_segments = []
    for i in range(n_replicates):
        start = i * frames_per_rep + trim_frames
        end = (i + 1) * frames_per_rep
        trimmed_segments.append(data[start:end])

    trimmed_data = np.concatenate(trimmed_segments, axis=0)
    return trimmed_data


In [ ]:
# Load apo and ATP trajs

apo_traj=md.load_dcd('path/to/joined.dcd',top='path/to/step3_input.pdb')
atp_traj=md.load_dcd('path/to/joined.dcd',top='path/to/step3_input.pdb')


In [ ]:
# Calculate distances

apo_dist=calc_com_distance(apo_traj)
atp_dist=calc_com_distance(atp_traj)

In [ ]:
# Trim distances

apo_dist=trim_replicate_timeseries(apo_dist,5000,10,300)
atp_dist=trim_replicate_timeseries(atp_dist,5000,10,300)

In [ ]:
# Plot KDE 

pl.figure(figsize=(8,6))



sb.kdeplot(apo_dist*10,label='Ligand-free',linewidth=4,color='darkgray',shade=False,alpha=1,bw_adjust=0.7)
sb.kdeplot(atp_dist*10,label='ATP-bound',linewidth=4,color='crimson',shade=False,alpha=0.8,bw_adjust=0.7)



pl.xlabel('')
pl.xlim(min(apo_dist*10),max(apo_dist*10))
pl.yticks([])
pl.ylabel('')
#pl.savefig('apo_atp_kde.pdf', format='pdf', dpi=300, bbox_inches='tight')

In [ ]:

fig, (ax1, ax2, ax3,ax4) = pl.subplots(1, 1, sharex=True,sharey=True, figsize=(12, 4))
ax1.plot(atp_dist[0:400]*10,linewidth=6,color='crimson')
ax1.plot(apo_dist[0:400]*10,linewidth=6,color='darkgrey',alpha=0.8)


pl.xlim(0,400)

pl.savefig('timeseries.png', format='png', dpi=300, bbox_inches='tight')